In [19]:
import numpy as np
import pandas as pd

import os, gc, warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.rc['font.size'] = 9.0
matplotlib.rc('font', size = 20)
matplotlib.rc('axes', titlesize = 20)
matplotlib.rc('axes', labelsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)
matplotlib.rc('legend', fontsize = 20)
matplotlib.rc('figure', titlesize = 20)

import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
init_notebook_mode(connected=True)
from IPython.display import display
from IPython.core.display import HTML
%matplotlib inline






In [20]:
countryCode_toName = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zealand",
    162: "Phillipines",
    166: "Qatar",
    184: "Singapore",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "UAE",
    215: "United Kingdom",
    216: "United States",
}

In [18]:
data = pd.read_csv("datafile/zomato.csv", encoding = "ISO-8859-1")
data['Country'] = data['Country Code'].apply(lambda x: countryCode_toName[x])
data.sample(5)

Restaurant ID   Restaurant Name  Country Code       City  \
1808       18466973          Fruitpro             1    Gurgaon   
8016       18478987     Sher E Punjab             1  New Delhi   
6007         300608      Chowmein Hut             1  New Delhi   
8374       18367439  Hooting Owl Cafe             1      Noida   
3199         311511           Giani's             1  New Delhi   

                                            Address        Locality  \
1808     Unit Sez Aminity Block, Sector 21, Gurgaon       Sector 21   
8016  73, Gautam Nagar Road, Yusuf Sarai, New Delhi     Yusuf Sarai   
6007        1346, Sangarashan, Paharganj, New Delhi       Paharganj   
8374           97, B Block Market, Sector 15, Noida       Sector 15   
3199             Shop 41, Defence Colony, New Delhi  Defence Colony   

               Locality Verbose  Longitude   Latitude              Cuisines  \
1808         Sector 21, Gurgaon  77.071691  28.510065             Beverages   
8016     Yusuf Sarai, New Delhi  77.208147  28.560930          North Indian   
6007       Paharganj, New Delhi  77.212264  28.643792               Chinese   
8374           Sector 15, Noida  77.310141  28.582123  Fast Food, Beverages   
3199  Defence Colony, New Delhi  77.230591  28.573498   Ice Cream, Desserts   

       ...    Has Table booking Has Online delivery Is delivering now  \
1808   ...                   No                  No                No   
8016   ...                   No                  No                No   
6007   ...                   No                  No                No   
8374   ...                   No                  No                No   
3199   ...                   No                 Yes                No   

     Switch to order menu Price range Aggregate rating  Rating color  \
1808                   No           1              0.0         White   
8016                   No           1              0.0         White   
6007                   No           1              2.9        Orange   
8374                   No           1              0.0         White   
3199                   No           1              2.8        Orange   

      Rating text Votes Country  
1808    Not rated     0   India  
8016    Not rated     1   India  
6007      Average     4   India  
8374    Not rated     2   India  
3199      Average    26   India  

[5 rows x 22 columns]

In [32]:
#Zomato's Presence across the world
label = list(data.Country.value_counts().index)
values = list(data.Country.value_counts().values)

fig = {
    "data":[
        {
            "labels" : label,
            "values" : values,
            "hoverinfo" : 'label+percent',
            "domain": {"x": [0, .9]},
            "hole" : 0.6,
            "type" : "pie",
            "rotation":120,
        },
    ],
    "layout": {
        "title" : "Zomato's Presence around the World",
        "annotations": [
            {
                "font": {"size":30},
                "showarrow": True,
                "text": "Countries",
                "x":0.2,
                "y":0.9,
            },
        ]
    }
}

iplot(fig)

In [44]:
data_india = data.loc[data.Country == "India"]
data_india['Text'] = data_india['Restaurant Name'] + "<br>" + data_india['Locality Verbose']
data_india_rest = data_india[['Restaurant Name','Aggregate rating',
                              'Average Cost for two']].groupby('Restaurant Name').mean()


In [71]:
data = [
    go.Scatter(x = data_india_rest['Average Cost for two'],
              y = data_india_rest['Aggregate rating'],
               text = data_india['Text'],
              mode = "markers",
              marker = dict(opacity = 0.7,
                           size = 10,
                           color = data_india_rest['Aggregate rating'],
                           colorscale = 'Viridis',
                           showscale = True,
                            maxdisplayed = 2500,
                           ),
              hoverinfo="text+x+y",
              )
]

layout = go.Layout(autosize = True,
                  xaxis = dict(title = "Average Cost of Two (INR)",
                              #titlefont=dict(size=20,),
                             #tickmode="linear",
                              ),
                   yaxis = dict(title = "Rating",
                               #titlefont = dict(size=17,),
                               ),
                  )
iplot(dict(data = data, layout = layout))

In [72]:
ncr_data = data_india.loc[data_india.City.isin(['New Delhi','Gurgaon','Noida','Faridabad'])]

In [83]:
x_ax = ncr_data.City.value_counts().index
y_ax = ncr_data.City.value_counts().values

data = [
    go.Bar(x = x_ax,
          y = y_ax,
          text = y_ax,
          textposition='auto',
          marker = dict(color = 'rgb(80, 228, 188)',
                       line = dict(color='rgb(8, 48, 107)',
                                  width=1.5),
                       ),
          opacity=0.6,
        hoverinfo="none"
          )
]

layout = go.Layout(title = "Number of Restaurants Across Major Cities",
                   yaxis = dict(title = "Number of Restaurants/Eateries <br> (Associated with Zomato)"),
                   xaxis = dict(title="Cities",
                               titlefont=dict(size=30),),
                  )


fig = go.Figure(data=data, layout=layout)

iplot(fig)


In [84]:
#
types = {
    "Breakfast and Coffee" : ["Cafe Coffee Day", "Starbucks", "Barista", "Costa Coffee", "Chaayos",
                             "Dunkin' Donuts"],
    "American" : ["Domino's Pizza", "McDonald's", "Burger King", "Subway", "Dunkin' Donuts", "Pizza Hut"],
    "Ice Creams and Shakes" : ["Keventers", "Giani", "Giani's", "Starbucks", "Baskin Robbins", "Nirula's Ice Cream"]
}
breakfast = ncr_data.loc[ncr_data['Restaurant Name'].isin(types['Breakfast and Coffee'])]
american = ncr_data.loc[ncr_data['Restaurant Name'].isin(types['American'])]
ice_cream = ncr_data.loc[ncr_data['Restaurant Name'].isin(types['Ice Creams and Shakes'])]

print ("Breakfast: ", breakfast.shape, "\nFast Food: ", american.shape, "\nIce Cream: ", ice_cream.shape)

('Breakfast: ', (178, 23), '\nFast Food: ', (238, 23), '\nIce Cream: ', (138, 23))


In [91]:
#Common places to have coffee and breakfast including Starbuck,Barista etc
breakfast_rating = breakfast[['Restaurant Name',
                             'Aggregate rating']].groupby('Restaurant Name').mean().reset_index().sort_values('Aggregate rating', ascending = False)
x_ax = breakfast_rating['Restaurant Name']
y_ax = breakfast_rating['Aggregate rating'].apply(lambda x:round(x, 2))

data = [
    go.Bar(x = x_ax,
          y = y_ax,
          text = y_ax,
          textposition = 'auto',
          marker = dict(color = 'rgb(159, 202, 220)',
                       line = dict(color = 'rgb(8, 48, 107)',
                                  width = 1.5),
                       ),
          opacity = 0.6,
          hoverinfo = "none"
          )
]

layout = go.Layout(title = "Average Ratings : Breakfast & Coffee",
                   yaxis = dict(title = "Average Rating",
                               titlefont = dict(size = 20)),
                   xaxis = dict(title="Cafe",
                               titlefont=dict(size=20),),
                  )


fig = go.Figure(data=data, layout=layout)

iplot(fig)

